## Unspoiled - Phantom Tracker
#### Ben Cobb

Spring 2024

## Imports and Libraries

In [2]:
import cv2 as cv
import os
import time
import datetime
import requests
from PIL import Image
from matplotlib import pyplot as plt

# Global Variables

In [126]:

camIDs = [1, 2, 3]     # Built-in webcam is ID 0. We can change these values to whatever our three camera's ID is
camDepts = [0, 1, 2]   # This is to track which camera is covering which department
itemID = 4
qty = 2
add = True

classes = ["MILK", "EGGCARTON", "CREAMER", "APPLE", "BANANA", "PEAR", "COUGHDROPS", "CHEEZIT", "SODA"]
'''
classIDs:
0 = "milk"
1 = "eggcarton"
2 = "creamer"
3 = "apple"
4 = "banana"
5 = "pear"
6 = "coughdrops"
7 = "cheezit"
8 = "soda"
'''


##############
# CONNECTION #
##############
ip = "74.117.171.112"
port = "32008"
connURL = f"http://{ip}:{port}"

########
# URLS #
########
checkDeptURL = f"{connURL}/Inventory/GetInventoryItem/"
checkPhantomURL = f"{connURL}/Phantom/GetAllPhantomsByCenter/"
createPhantomURL = f"{connURL}/Phantom/CreatePhantom/"

#########
# OTHER #
#########
interval = 3    # how many seconds to wait before taking another pic
boxCount = 0
cmap = "gray"   # for displaying images

# Methods

In [103]:
# Checks the item's department, returns true if matches camDeptID
def getInv(classID, camDeptID):

    match = False
    deptID = -1
        
    # Attempt a connection
    try:
        response = requests.get(f"{checkDeptURL}{str(classID)}")    
        statusCode = response.status_code
        
        # We're in!
        if statusCode == 200:
            
            # Parses the response JSON
            data = response.json()
    
            # If the ID doesn't exist
            if not data["success"]:
                print("NO SUCH ITEM!")
                deptID = -1
                
            # If the ID DOES exist
            else:
                deptID = data["inventory"]["deptId"]
                if camDeptID == deptID:   # Item is in correct department
                    match = True
                    
        else:
            print("CONNECTION ERROR: ", statusCode)
            deptID = -2
               
    except:
        print("UNEXPECTED CONNECTION ERROR!!!")
        print("Possibly the server isn't open?")
        deptID = -3
        
    return match, deptID

In [106]:
# Checks the PHANTOMS table to see if it's already been logged, returns true if so
def checkPhantom(classID, camDeptID, centerX, centerY):

    match = False
    deptID = -1
    
    # Calculates the range of the center coords
    epsilon = .05
    centerXMin = str(round(centerX - .05, 6))
    centerXMax = str(round(centerX + .05, 6))
    centerYMin = str(round(centerY - .05, 6))
    centerYMax = str(round(centerY + .05, 6))
        
    # Attempts a connection
    try:
        response = requests.get(f"{checkPhantomURL}{centerXMin}/{centerXMax}/{centerYMin}/{centerYMax}")
        statusCode = response.status_code
        
        # We're in!
        if statusCode == 200:
            
            # Parses the response JSON
            data = response.json()
    
            # If the ID doesn't exist
            if not data["success"]:
                print("NO SUCH ITEM!")
                deptID = -1
                
            # If PHANTOM DOES exist
            else:
                
                # Go through list of returned PHANTOMS to see if there's one that matches the itemID and departmentID
                phantoms = data["phantoms"] # ?
                for i in range(len(phantoms)):
                    
                    deptID = phantoms[i]["deptCurrent"]
                    if camDeptID == deptID:   # Item is in correct department
                        match = True
                        
        else:
            print("CONNECTION ERROR: ", statusCode)
            deptID = -2
                   
    except:
        print("UNEXPECTED CONNECTION ERROR!!!")
        print("Possibly the server isn't open?")
        deptID = -3

    return match, deptID

In [141]:
# Creates a Phantom for the PHANTOMS table, return True if successful
def createPhantom(classID, camDeptID, centerX, centerY, width, height, img):

    inserted = False

    # Gets the filepath where the box-cropped img is saved
    # Or returns the box-cropped img itself 
    # :shrug:
    boxImg = saveBox(centerX, centerY, width, height, img)  # If it's this one, we need to figure out how to BLOB!
    imgDir = saveBox(centerX, centerY, width, height, img)
    
    phantom = {
        "centerX": centerX,
        "centerY": centerY,
        "claimed": False,
        "deptCurrent": camDeptID,
        "empId": 0, 
        "height": height,
       # "imgDir": imgDir,
        "invId": classID,
        "phantomId": 0,
        "returned": False,
        "timeFound": datetime.datetime.now().isoformat(),
        "width": width
    }
    
    # Attempts a connection
    try:
        response = requests.post(createPhantomURL, json=phantom)
        statusCode = response.status_code
        
        # We're in!
        if response.status_code == 200:
            inserted = True
        else:
            print(f"ERROR -- STATUS CODE: {status_code}")

    except:
        print("UNEXPECTED CONNECTION ERROR!!!")
        print("Possibly the server isn't open?")
    
    return inserted

In [134]:
# Crops image to relevant box and returns it as an img
def saveBox(centerX, centerY, width, height, img):

    boxImgPath = f"./images_OLD/box-({centerX},{centerY}).png"
    
    # Gets image width and height
    imgWidth, imgHeight = img.size

    # Calculate pixel coordinates from centerX, centerY, width, and height
    minX = int((centerX - width / 2) * imgWidth)
    minY = int((centerY - height / 2) * imgHeight)
    maxX = int((centerX + width / 2) * imgWidth)
    maxY = int((centerY + height / 2) * imgHeight)

    # Crops the image to just the area of the bounding box
    boxImg = img.crop((minX, minY, maxX, maxY))
    plt.imshow(boxImg, cmap)
    
    # Saves the cropped image to the boxImgPath
    boxImg.save(boxImgPath)

    return boxImgPath

# Insert Items

In [146]:
# Get item from Inventory API






0: 384x640 1 milk, 2 eggcartons, 1 apple, 4 bananas, 1 pear, 1 cheezit, 64.4ms
Speed: 1.5ms preprocess, 64.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)


# Delete Items

In [146]:
# Runs a picture through the model

# Temp path for testing
camImgPath = './images_OLD/groupshot10000.png'
camDeptID = 2   # This will change, as we'll loop through the three cameras

camImg = Image.open(camImgPath)
results = model(camImg) # The camera's picture goes here!


0: 384x640 1 milk, 2 eggcartons, 1 apple, 4 bananas, 1 pear, 1 cheezit, 64.4ms
Speed: 1.5ms preprocess, 64.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
